<a href="https://colab.research.google.com/github/SalahKhattab74/WordEmbeddingNLP/blob/main/WordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf   

In [3]:
print(tf.__version__)

2.7.0


In [4]:
import tensorflow_datasets as tfds 
imdb , info = tfds.load("imdb_reviews", with_info = True , as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHJC1QW/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteHJC1QW/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
import numpy as np 
train_data , test_data = imdb['train'],imdb['test']

In [6]:
training_sentences =[]
training_labels =[]
testing_sentences=[]
testing_labels=[]

for s , l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s , l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [9]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [12]:
from tensorflow.keras.preprocessing import sequence
vocab_size = 10000
embedding_dim = 16
max_length = 120 
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)


testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [14]:
#Model 
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [18]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))


Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.4962 - accuracy: 0.7390 - val_loss: 0.3502 - val_accuracy: 0.8475
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2490 - accuracy: 0.9036 - val_loss: 0.3679 - val_accuracy: 0.8412
Epoch 3/10
782/782 [==============================] - 5s 7ms/step - loss: 0.1119 - accuracy: 0.9695 - val_loss: 0.4341 - val_accuracy: 0.8281
Epoch 4/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0319 - accuracy: 0.9952 - val_loss: 0.5188 - val_accuracy: 0.8273
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0120 - accuracy: 0.9986 - val_loss: 0.5895 - val_accuracy: 0.8222
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0050 - accuracy: 0.9994 - val_loss: 0.6253 - val_accuracy: 0.8261
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.6810 - val_accuracy: 0.8243
Epoch 

In [19]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 16)


In [22]:
reverse_word_index = dict([(value,key)for (key,value) in word_index.items()])
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


In [23]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>